# WhisperTS joining JSON to SRT

In [16]:
import re,codecs,unidecode
import pandas as pd
import networkx as nx
import matplotlib.pylab as p

import json,glob,os
nMachine=os.uname()[1]
if "iMac" in nMachine:
    nMachine="iMac"
elif "MacBook" in nMachine:
    nMachine="MacBook"
print(nMachine)

iMac


In [17]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

In [18]:
repJson="/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/%s/Json/"%nMachine
segments=len(glob.glob(repJson+"*.json"))
segments

9

In [19]:
json1=repJson+"extrait-00.json"
with open(json1) as inFile:
    whisper1=json.load(inFile)

### Gestion des timestamps

In [20]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

def shiftTime(timestamp,decal):
    return second2timestamp(timestamp2second(timestamp)+decal)

In [21]:
punct = list('''!()[]{};:-'"\,<>./?@#$%^&*_~''')
maxSize=8

def removePunct(texte):
    result=texte
    for p in punct:
        result=result.replace(p,"")
    return result

### Gestion des structures JSON de Whisper

In [22]:
def getWords(whisper):
    lWords=[]
    for segment in whisper["segments"]:
        for word in segment["words"]:
            lWords.append((second2timestamp(word["start"]),
                              second2timestamp(word["end"]),
                              word["text"],
                              word["confidence"]))        
    return [(t1,t2,w,c) for t1,t2,w,c in lWords if w!=""]

def getList(tWhisper):
    return [unidecode.unidecode(removePunct(w)).lower() for _,_,w,_ in tWhisper]

In [23]:
srt=getWords(whisper1)
srtList=getList(srt)

### Trouver la jointure

In [24]:
def findSample(sample,ref):
    lowSample=[t.lower() for t in sample]
    lowRef=[t.lower() for t in ref]
    i=0
    while i<len(ref)-len(sample):
        # print("ref",i)
        if lowSample==lowRef[i:i+len(sample)]:
            return i
        else:
            i+=1
    return None

### Collage des mots
1. on cherche la partie commune entre le nouveau segment et les sous-titres précédents
2. on peut régler la longueur de la partie commune pour optimiser le collage *maxSize*
3. on incrémente les temps de la nouvelle partie à partir du deuxième mot (pour éviter les erreurs de timestamp en début d'enregistrement)

In [25]:
def pasteWords(srt,whisper2):
    srtList=getList(srt)
    w2=getWords(whisper2)
    w2List=getList(w2)
    lenW2=len(w2)
    compSize=min(maxSize,lenW2)
    i=1
    print(compSize)
    while i<lenW2-compSize:
        sample=w2List[i:i+compSize]
        ref=srtList
        # print(sample)
        idx=findSample(sample,ref)
        if idx:
            print("found",i,idx)
            break
        else:
            i+=1
    if idx:
        print("found",i,idx,srtList[idx],srt[idx][0],w2List[i],w2[i][0])
        tDecal=timestamp2second(srt[idx+1][0])-timestamp2second(w2[i+1][0])
        print(second2timestamp(tDecal))
        newSrt=[]
        for n in range(i+1,lenW2):
            nWord=w2[n]
            newWord=(shiftTime(nWord[0],tDecal),shiftTime(nWord[1],tDecal),nWord[2],nWord[3])
            newSrt.append(newWord)
        print(srt[idx+1],newSrt[0])
        return srt[:idx]+newSrt
    else:
        print("problème")
        ding()
        ding()
        ding()
        return None

In [26]:
if segments:
    for i in range(1,segments):
        print(i)
        json2=repJson+"extrait-%02d.json"%i
        with open(json2) as inFile:
            whisper2=json.load(inFile)
        newSrt=pasteWords(srt,whisper2)
        if not newSrt:
            break
        else:
            srt=newSrt
# print(srt)

1
8
found 1 128
found 1 128 vraiment 00:01:00,420 vraiment 00:00:00,420
00:01:00,100
('00:01:00,640', '00:01:00,820', 'un', 0.993) ('00:01:00,640', '00:01:00,899', 'un', 0.986)
2
8
found 55 501
found 55 501 regarde 00:02:46,979 regarde 00:00:46,859
00:02:00,098
('00:02:47,159', '00:02:47,219', 'la', 0.967) ('00:02:47,158', '00:02:47,217', 'la', 0.965)
3
8
found 12 552
found 12 552 ouais 00:03:02,598 ouais 00:00:02,250
00:03:00,188
('00:03:02,738', '00:03:02,858', 'là', 0.512) ('00:03:02,737', '00:03:02,959', 'là', 0.469)
4
8
found 34 780
found 34 780 le 00:04:13,127 le 00:00:13,339
00:04:00,147
('00:04:13,627', '00:04:13,868', 'blanc', 0.972) ('00:04:13,627', '00:04:13,867', 'blanc', 0.988)
5
8
found 168 1138
found 168 1138 il 00:05:53,056 il 00:00:52,570
00:05:00,528
('00:05:53,397', '00:05:53,556', 'va', 0.931) ('00:05:53,397', '00:05:53,897', 'va', 0.969)
6
8
found 17 1287
found 17 1287 cest 00:06:33,778 cest 00:00:33,200
00:06:00,518
('00:06:33,978', '00:06:34,057', 'pas', 0.977) (

In [27]:
pause=0.200

srtTours=[]
srtMots=[]
bTour=True
prevEnd=0
pStart=second2timestamp(0)
pEnd=0

for i,(start,end,mot,confidence) in enumerate(srt):
    # print(start,end,mot,confidence)
    tStart=timestamp2second(start)
    tEnd=timestamp2second(end)
    # print(start,tStart,prevEnd)
    if tStart-prevEnd>pause and prevEnd!=0:
        pEnd=second2timestamp(prevEnd)
        pTour=" ".join(srtMots)
        srtTours.append((pStart,pEnd,pTour))
        pStart=start
        prevEnd=tEnd
        srtMots=[mot]
        bPhrase=True
    else:
        srtMots.append(mot)
        prevEnd=tEnd
pTour=" ".join(srtMots)
srtTours.append((pStart,pEnd,pTour))

In [28]:
ding()

In [29]:
srtLines=[]
for i,(start,end,mot) in enumerate(srtTours):
    print(start,end,mot)
    n=i+1
    srtLines.append(str(n))
    srtLines.append(start+" --> "+end)
    srtLines.append(mot)
    srtLines.append("")


00:00:00,000 00:00:18,760 PICTOPLAYNER Kana,
00:00:19,100 00:00:22,879 qui est en train de [*] s'installer, notre magnifique jungler, Closer,
00:00:23,100 00:00:25,559 qui lui aussi est en train de s'installer, Rhea derrière,
00:00:25,780 00:00:33,759 [*] les [*] derniers petits réglages, les derniers petits détails, le setup avant [*] de s'envoyer le génie Vladi,
00:00:34,100 00:00:34,399 [*]
00:00:34,729 00:00:37,880 le génie Vladi qui a fait vraiment deux [*] très bonnes games,
00:00:38,219 00:00:47,420 je trouve moi, pour ses débuts en LEC, franchement il a été [*] au niveau, notre hyper carry, [*] notre Jinx, [*] F7,
00:00:47,719 00:00:49,420 [*] qui a été chaud hier aussi,
00:00:50,520 00:00:51,420 il a vraiment [*] fait une game de fou,
00:00:52,259 00:00:53,899 et Taragou, même s'il a eu un early difficile,
00:00:54,219 00:01:00,420 sur la fin de la game, le mid game et tout, il a sorti des combos, il a toujours été à l'initiation, avec Alistar, c'est
00:01:00,640 00:01:02,179 

In [30]:
with codecs.open(json1.replace(".json","-tours.srt"),"w",encoding="utf8") as outFile:
    for srtLine in srtLines:
        outFile.write(srtLine+"\n")